In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
import folium
from sklearn import preprocessing
import geopandas as gpd
import matplotlib.colors as colors
import matplotlib.colors as colors
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
#from helpers import *
from ipywidgets import SelectMultiple, RadioButtons, interact, Layout
%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objects as go
import plotly.offline as off

In [3]:
import networkx as nx

## Interactive Plotly
Now we are going to use plotly interactively so that we can choose the country we want to plot.

In [4]:
# Load the dataset
mat = pd.read_pickle("pickle/trade_matrix.pkl")
matn = pd.read_pickle("pickle/trade_matrix_norm.pkl")
val = pd.read_pickle("pickle/value_production.pkl")

In [5]:
# Define the countries list
years = ['Y'+str(2000+x) for x in range(0,17)]
tot_prod = val[years].mean(axis=1)
#list_countries = tot_prod.sort_values(ascending=False).index.drop('World')
list_countries = tot_prod.index.drop('World').sort_values()

### Function for retreiveng the data

In [6]:
def networkgraph(country, matn, mat, val, n, mode):
    # Retreive the total mean production value for the country
    years = ['Y'+str(2000+x) for x in range(0,17)]
    tot_prod = val[years].mean(axis=1).loc[country]
    
    if mode == 'export':
        # Select the country and do the mean
        exportn = matn[(matn.reset_index().From==country).values]
        exportn = exportn.mean(axis=1)
        exportn = exportn.reset_index().set_index(['From', 'To'])
        exportn = exportn.rename(columns={0:"Value"})
        # Do the same for the absolute values
        export = mat[(mat.reset_index().From==country).values]
        export = export.mean(axis=1)
        export = export.reset_index().set_index(['From', 'To'])
        export = export.rename(columns={0:"Value"})
        

        # Select the top n
        n = 30
        n = n-1
        exportn = exportn.sort_values(['Value'], ascending=False)
        topnexn = exportn.iloc[:n]
        countries = set(topnexn.reset_index().From.values).union(topnexn.reset_index().To.values)

        # Retrieve these countries' absolute values
        topnex = export.reset_index().set_index('To').loc[list(countries)]

        # Add the nodes to the network
        G = nx.DiGraph()
        G.add_nodes_from(countries)
        # Add the edges
        G.add_weighted_edges_from(topnexn.reset_index().values)

        # Retreive the export values
        datan = topnexn.Value.reset_index().drop('From', axis=1).set_index('To')
        ton = pd.Series(datan['Value'], index=datan.index)
        try:
            ton[country]
        except:
            ton = ton.append(pd.Series([0], index=[country]))
        max_ton=max(ton)

        # Do the same for the absolute values
        data = topnex.Value.reset_index().set_index('To')
        to = pd.Series(data['Value'], index=data.index)
        try:
            to[country]
        except:
            to = to.append(pd.Series([0], index=[country]))

        # Add the import values as attributes to the nodes
        nx.set_node_attributes(G, ton, 'To_norm')
        nx.set_node_attributes(G, to, 'To')
        pos = nx.spring_layout(G)
        nx.set_node_attributes(G, pos, 'pos')
        
        return G, max_ton, tot_prod;
        
    else:
        # Select the country and do the mean
        imprtn = matn[(matn.reset_index().To==country).values]
        imprtn = imprtn.mean(axis=1)
        imprtn = imprtn.reset_index().set_index(['From', 'To'])
        imprtn = imprtn.rename(columns={0:"Value"})
        # Do the same for the absolute values
        imprt = mat[(mat.reset_index().To==country).values]
        imprt = imprt.mean(axis=1)
        imprt = imprt.reset_index().set_index(['From', 'To'])
        imprt = imprt.rename(columns={0:"Value"})

        # Select the top n
        imprt = imprt.sort_values(['Value'], ascending=False)
        topnim = imprt.iloc[:n]
        countries = set(topnim.reset_index().From.values).union(topnim.reset_index().To.values)

        # Retrieve these countries' normalized values
        topnimn = imprtn.reset_index().set_index('From').loc[list(countries)]

        # Add the nodes to the network
        G = nx.DiGraph()
        G.add_nodes_from(countries)
        # Add the edges
        G.add_weighted_edges_from(topnim.reset_index()[['To', 'From', 'Value']].values)

        # Retreive the import values
        data = topnim.Value.reset_index().drop('To', axis=1).set_index('From')
        frm = pd.Series(data['Value'], index=data.index)
        try:
            frm[country]
        except:
            frm = frm.append(pd.Series([0], index=[country]))
        max_frm=max(frm)

        # Do the same for the normalized values
        datan = topnimn.Value.reset_index().set_index('From')
        frmn = pd.Series(datan['Value'], index=datan.index)
        try:
            frmn[country]
        except:
            frmn = frmn.append(pd.Series([0], index=[country]))


        # Add the import values as attributes to the nodes
        nx.set_node_attributes(G, frmn, 'From_norm')
        nx.set_node_attributes(G, frm, 'From')
        pos = nx.spring_layout(G)
        nx.set_node_attributes(G, pos, 'pos')

        return G, max_frm, tot_prod;

### Export

In [ ]:
fig = go.Figure()

for country in list_countries:
    G, max_ton, tot_prod = networkgraph(country, matn, mat, val, 30, 'export')
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines',
        visible = False)

    node_x = []
    node_y = []
    labels = []

    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
        if node != country:
            labels.append(node)
        else:
            labels.append('<b>' + node + '</b>')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers + text',
        hoverinfo='text',
        text = labels,
        textfont=dict(
            family="Arial",
            size=10,
            color= 'firebrick'
            #'#ff7f0e'
        ),
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=[],
            colorbar=dict(
                thickness=15,
                title='Percentage of ' + country + '\'s total production',
                xanchor='left',
                titleside='right'
            ),
            line_width=2),
        visible = False)

    node_color = []
    node_text = []
    node_size = []
    for node in G.nodes():
        if node != country:
            node_text.append('Export value: ' + str(round(G.nodes()[node]['To_norm']*100, 4)) +'%<br>'
                             + 'Absolute value: ' + str (int(G.nodes()[node]['To']/1000)) + ' mln US$')
            node_color.append(G.nodes()[node]['To_norm'])
            node_size.append(np.log(1+G.nodes()[node]['To_norm'])*50/np.log(1+max_ton))

        else:
            node_text.append(country + '\'s total production: ' + str(int(tot_prod)) + ' mln US$')
            node_color.append('lightgray')
            node_size.append(50)
            node_textposition ='middle center',

    textposition="middle center",

    node_trace.marker.color = node_color
    node_trace.marker.size = node_size
    node_trace.hovertext = node_text

    fig.add_trace(edge_trace)
    fig.add_trace(node_trace)
    fig.update_layout(title='<b>Network graph of the export of:',
                      titlefont_size=16,
                      showlegend=False,
                      hovermode='closest',
                      margin=dict(b=20,l=5,r=5,t=40),
                      plot_bgcolor = 'white',
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    

C:\Users\Francesco\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: FutureWarning:


Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike



In [ ]:
buttons = []

buttons.append(dict(method='update',
                args=[{'visible': [False]*len(list_countries)*2}],
                label='Select Country'))

for i in range(0, len(list_countries)):
    show = [False]*len(list_countries)*2
    show[i*2] = True
    show[i*2+1] = True
    but = dict(method='update',
                args=[{'visible': show}],
                label=list_countries[i])
    buttons.append(but)

In [ ]:
# Here we create the menu using the buttons created above
menus=list([dict(x=0.5, y=1.10, yanchor='top', xanchor='left',  buttons = buttons)])

In [ ]:
fig.update_layout(updatemenus = menus) 

In [ ]:
# Save the representation
off.plot(fig, filename = 'overall_export.html', auto_open=False)

### Import

In [ ]:
#######################################################################################################################
fig = go.Figure()

for country in list_countries:
    G, max_frm, tot_prod = networkgraph(country, matn, mat, val, 30, 'import')
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines',
        visible = False)

    node_x = []
    node_y = []
    labels = []

    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
        if node != country:
            labels.append(node)
        else:
            labels.append('<b>' + node + '</b>')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers + text',
        hoverinfo='text',
        text = labels,
        textfont=dict(
            family="Arial",
            size=10,
            color= 'firebrick'
            #'#ff7f0e'
        ),
        marker=dict(
            showscale=True,
            # colorscale options
            #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
            #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
            #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=[],
            colorbar=dict(
                thickness=15,
                title = 'Imports of ' + country + ' absolute values in US $',
                xanchor='left',
                titleside='right'
            ),
            line_width=2),
        visible = False)

    node_color = []
    node_text = []
    node_size = []
    for node in G.nodes():
        if node != country:
            node_text.append('Perc. of ' + node + '\'s production: ' + str(round(G.nodes()[node]['From_norm']*100, 4)) +' %<br>'
                             + 'Absolute value: ' + str (int(G.nodes()[node]['From']/1000)) + ' mln US$')
            node_color.append(G.nodes()[node]['From']*1000)
            node_size.append(G.nodes()[node]['From']*50/max_frm)

        else:
            node_text.append(country + '\'s total production: ' + str(int(tot_prod)) + ' mln US$')
            node_color.append('lightgray')
            node_size.append(50)
            node_textposition ='middle center',

    textposition="middle center",

    node_trace.marker.color = node_color
    node_trace.marker.size = node_size
    node_trace.hovertext = node_text
    
    fig.add_trace(edge_trace)
    fig.add_trace(node_trace)
    fig.update_layout(title='<b>Network graph of the import of:',
                      titlefont_size=16,
                      showlegend=False,
                      hovermode='closest',
                      margin=dict(b=20,l=5,r=5,t=40),
                      plot_bgcolor = 'white',
                      xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                      yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
    
             

In [ ]:
buttons = []

buttons.append(dict(method='update',
                args=[{'visible': [False]*len(list_countries)*2}],
                label='Select Country'))

for i in range(0, len(list_countries)):
    show = [False]*len(list_countries)*2
    show[i*2] = True
    show[i*2+1] = True
    but = dict(method='update',
                args=[{'visible': show}],
                label=list_countries[i])
    buttons.append(but)

In [ ]:
# Here we create the menu using the buttons created above
menus=list([dict(x=0.5, y=1.10, yanchor='top', xanchor='left',  buttons = buttons)])

In [ ]:
fig.update_layout(updatemenus = menus) 

In [ ]:
# Save the representation
off.plot(fig, filename = 'overall_import.html', auto_open=False)